In [4]:
import sys
import gym
import numpy as np
from collections import defaultdict
import matplotlib.style
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')


In [5]:
env = gym.make('Blackjack-v0')
print(env.observation_space)

Tuple(Discrete(32), Discrete(11), Discrete(2))


### In model free prediction we do not have knowledge of the dynamic of the MDP, reward function etc. Given a policy we want to estimate value function of that policy.

### Monte Carlo learning is used in episodic MDP's i.e it learns from cmoplete episodes. The value function is estimated by sampling returns of being in a state and following policy 'PI' over many episodes.

In [4]:
def mc_prediction(policy, env, num_episodes, discount_factor=1.0):
    """
    Monte Carlo prediction algorithm. Calculates the value function
    for a given policy using sampling.
    
    Args:
        policy: A function that maps an observation to action probabilities.
        num_episodes: Number of episodes to sample.
        discount_factor: Gamma discount factor.
    
    Returns:
        A dictionary that maps from state -> value.
        The state is a tuple and the value is a float.
    """
    """
    1) Initialize a random policy 'PI'<------ To be evaluated
    2) Initialize arbitray state value function v('PI') under the policy
    3) Generate Episodes under policy 'PI' store in an array. An episode is a tuple of <state,action,reward>
    #NOTE: Action is chosen such that it follows policy 'PI'
    4) For each episode increment N(s)<---- N(s)+1 for every first visit to state s for each episode.
    5) Calculate total return for each episode starting from state s S(s)<----- S(s)+G_t
    #This is return following first occurance of state s in each episode
    6) Average these returns
    """

    # Keeps track of sum and count of returns for each state
    # to calculate an average. We could use an array to save all
    # returns (like in the book) but that's memory inefficient.
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)
    
    # The final value function
    V = defaultdict(float)
    for i_episode in range(1, num_episodes + 1):
        # Print out which episode we're on, useful for debugging.
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()

        #STEP 3 Generating episodes under policy 'PI'
        # Generate an episode.
        # An episode is an array of (state, action, reward) tuples
        episode = []
        state = env.reset()
        for t in range(100):
            action = policy(state)
            next_state, reward, done, _ = env.step(action)
            episode.append((state, action, reward))
            if done:
                break
            state = next_state
    
        states_in_episode = set([tuple(x[0]) for x in episode])
        for state in states_in_episode:
            # STEP 4: Find the first occurance of the state in each episode
            first_occurence_idx = next(i for i,x in enumerate(episode) if x[0] == state)
            # Sum up all rewards since the first occurance
            G = sum([x[2]*(discount_factor**i) for i,x in enumerate(episode[first_occurence_idx:])])
            # Calculate average return for this state over all sampled episodes
            returns_sum[state] += G
            returns_count[state] += 1.0
            V[state] = returns_sum[state] / returns_count[state]

    return V


In [5]:
def sample_policy(observation):
    """
    A policy that sticks if the player score is >= 20 and hits otherwise.
    """
    score, dealer_score, usable_ace = observation
    return 0 if score >= 20 else 1

# Monte Carlo Control Epsilon-Greedy

#### Policy Improvement is done by making policy greedy wrt to estimated action value function that is obtained by Policy Evaluation Step. We store the Q value in each state during policy evaluation step, which gives us the value function of taking every possible action from that state. Which is then used to find the greedy action that maximum Q value possible from that state. And policy is changed by taking that action which maximizes the Q value

In [6]:
def make_epsilon_greedy_policy(Q, epsilon, nA):
    """
    Creates an epsilon-greedy policy based on a given Q-function and epsilon.
    
    Args:
        Q: A dictionary that maps from state -> action-values.
            Each value is a numpy array of length nA (see below)
        epsilon: The probability to select a random action . float between 0 and 1.
        nA: Number of actions in the environment.
    
    Returns:
        A function that takes the observation as an argument and returns
        the probabilities for each action in the form of a numpy array of length nA.
    
    """
    def policy_fn(observation):
        A = np.ones(nA, dtype=float) * epsilon / nA
        best_action = np.argmax(Q[observation])
        A[best_action] += (1.0 - epsilon)
        return A
    return policy_fn

In [7]:
def mc_control_epsilon_greedy(env, num_episodes, discount_factor=1.0, epsilon=0.1):
    """
    Monte Carlo Control using Epsilon-Greedy policies.
    Finds an optimal epsilon-greedy policy.
    
    Args:
        env: OpenAI gym environment.
        num_episodes: Number of episodes to sample.
        discount_factor: Gamma discount factor.
        epsilon: Chance the sample a random action. Float betwen 0 and 1.
    
    Returns:
        A tuple (Q, policy).
        Q is a dictionary mapping state -> action values.
        policy is a function that takes an observation as an argument and returns
        action probabilities
    """
    
    # Keeps track of sum and count of returns for each state
    # to calculate an average. We could use an array to save all
    # returns (like in the book) but that's memory inefficient.
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)
    
    # The final action-value function.
    # A nested dictionary that maps state -> (action -> action-value).
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    
    # The policy we're following
    policy = make_epsilon_greedy_policy(Q, epsilon, env.action_space.n)
    
    for i_episode in range(1, num_episodes + 1):
        # Print out which episode we're on, useful for debugging.
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()

        # Generate an episode.
        # An episode is an array of (state, action, reward) tuples
        episode = []
        state = env.reset()
        for t in range(100):
            probs = policy(state)
            action = np.random.choice(np.arange(len(probs)), p=probs)
            next_state, reward, done, _ = env.step(action)
            episode.append((state, action, reward))
            if done:
                break
            state = next_state

        # Find all (state, action) pairs we've visited in this episode
        # We convert each state to a tuple so that we can use it as a dict key
        sa_in_episode = set([(tuple(x[0]), x[1]) for x in episode])
        for state, action in sa_in_episode:
            sa_pair = (state, action)
            # Find the first occurance of the (state, action) pair in the episode
            first_occurence_idx = next(i for i,x in enumerate(episode)
                                       if x[0] == state and x[1] == action)
            # Sum up all rewards since the first occurance
            G = sum([x[2]*(discount_factor**i) for i,x in enumerate(episode[first_occurence_idx:])])
            # Calculate average return for this state over all sampled episodes
            returns_sum[sa_pair] += G
            returns_count[sa_pair] += 1.0
            Q[state][action] = returns_sum[sa_pair] / returns_count[sa_pair]
        
        # The policy is improved implicitly by changing the Q dictionary
    
    return Q, policy

In [8]:
Q, policy = mc_control_epsilon_greedy(env, num_episodes=500000, epsilon=0.1)


Episode 500000/500000.

In [11]:
V = defaultdict(float)
for state, actions in Q.items():
    action_value = np.max(actions)
    V[state] = action_value
plt.plot_value_function(V, title="Optimal Value Function")

AttributeError: module 'matplotlib.pyplot' has no attribute 'plot_value_function'

In [9]:
V_10k = mc_prediction(sample_policy, env, num_episodes=10000)
plt.plot_value_function(V_10k, title="10,000 Steps")

V_500k = mc_prediction(sample_policy, env, num_episodes=500000)
plt.plot_value_function(V_500k, title="500,000 Steps")

Episode 10000/10000.

AttributeError: module 'matplotlib.pyplot' has no attribute 'plot_value_function'

#### Random policy check

In [22]:
for i_episode in range(5):
    state=env.reset()
    while True:
        print(state)
        action = env.action_space.sample()
        print('Stick') if action == 0 else print('hit')
        state, reward, done, info = env.step(action)
        if done:
            print('Reward', reward)
            print('You won') if reward > 0 else print('You lost')
            break

(12, 2, False)
Stick
Reward -1.0
You lost
(12, 8, False)
hit
(21, 8, False)
Stick
Reward 1.0
You won
(17, 10, False)
hit
Reward -1
You lost


## Monte Carlo Prediction of Action Value Function

#### Policy is stick when sum>18 and hit when sum<18 with 80% probability for both

In [4]:
def generate_episode(bj_env):
    episode=[]
    state= bj_env.reset()
    
    while True:
        
        probs = [0.8,0.2] if state[0]>18 else [0.2,0.8]
        action = np.random.choice(np.arange(2), p=probs)
        next_state, reward, done, info = bj_env.step(action)
        episode.append((state,action,reward))
        state= next_state
        
        if done:
            break
        
    return episode

In [5]:
for i in range(3):
    print(generate_episode(env))

[((13, 10, False), 1, -1)]
[((9, 10, False), 1, 0), ((16, 10, False), 1, 0), ((18, 10, False), 1, -1)]
[((15, 3, False), 0, -1.0)]


In [6]:
def action_val_pred(env, num_episodes, generate_episode, gamma=1.0):
    
    # initialize empty dictionaries of arrays
    returns_sum = defaultdict(lambda: np.zeros(env.action_space.n))
    
    N = defaultdict(lambda: np.zeros(env.action_space.n))
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    
    # loop over episodes
    for i_episode in range(1, num_episodes+1):
        
        # monitor progress
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
            
        # generate an episode
        episode = generate_episode(env)
        
        # obtain the states, actions, and rewards
        states, actions, rewards = zip(*episode)
        
        # prepare for discounting
        discounts = np.array([gamma**i for i in range(len(rewards)+1)])
        
        # update the sum of the returns, number of visits, and action-value 
        # function estimates for each state-action pair in the episode
        for i, state in enumerate(states):
            
            returns_sum[state][actions[i]] += sum(rewards[i:]*discounts[:-(1+i)])
            
            N[state][actions[i]] += 1.0
            Q[state][actions[i]] = returns_sum[state][actions[i]] / N[state][actions[i]]
            
    return Q

In [9]:
Q = action_val_pred(env, 500, generate_episode)


In [10]:
Q

defaultdict(<function __main__.action_val_pred.<locals>.<lambda>()>,
            {(20, 3, False): array([ 0.75, -1.  ]),
             (13, 10, False): array([-0.5       , -0.73684211]),
             (17, 10, False): array([-0.25      , -0.33333333]),
             (21, 10, False): array([ 0.72727273, -1.        ]),
             (14, 1, False): array([ 0.        , -0.66666667]),
             (8, 5, False): array([ 0., -1.]),
             (17, 5, False): array([ 0., -1.]),
             (17, 1, True): array([0., 0.]),
             (19, 1, True): array([0., 0.]),
             (21, 6, True): array([ 0., -1.]),
             (16, 6, False): array([-1., -1.]),
             (17, 7, False): array([ 0. , -0.2]),
             (21, 7, False): array([1., 0.]),
             (21, 1, True): array([ 0., -1.]),
             (12, 1, False): array([-1. , -0.8]),
             (16, 1, False): array([ 0., -1.]),
             (17, 1, False): array([-1., -1.]),
             (12, 2, True): array([ 0., -1.]),
    